# Exploring the BAM file format

In [2]:
import pysam 
import pandas as pd
import re
import numpy as np


## Read BAM file

In [3]:
file = pysam.AlignmentFile("./Oligo_1.bam", "rb")
print(file.__doc__)

AlignmentFile(filepath_or_object, mode=None, template=None,
    reference_names=None, reference_lengths=None, text=NULL,
    header=None, add_sq_text=False, check_header=True, check_sq=True,
    reference_filename=None, filename=None, index_filename=None,
    filepath_index=None, require_index=False, duplicate_filehandle=True,
    ignore_truncation=False, threads=1)

    A :term:`SAM`/:term:`BAM`/:term:`CRAM` formatted file.

    If `filepath_or_object` is a string, the file is automatically
    opened. If `filepath_or_object` is a python File object, the
    already opened file will be used.

    If the file is opened for reading and an index exists (if file is BAM, a
    .bai file or if CRAM a .crai file), it will be opened automatically.
    `index_filename` may be specified explicitly. If the index is not named
    in the standard manner, not located in the same directory as the
    BAM/CRAM file, or is remote.  Without an index, random access via
    :meth:`~pysam.AlignmentFile.fe

In [3]:
# get all reads (just printing the first 10 here)
i = 0
for read in file:
    print(read)
    i +=1 
    if i >= 10: break

print("\n")

# get read from a specific region
for read in file.fetch("oligo", 9, 10):
    print(read)


01670043-4149-45e8-882b-4d25ad62ee56	0	#0	9	3	13M1D13M1D21M3I2M1D37M	*	0	0	GATAGGTAGGACTTTTAGCTAGTGAACCTAGCCTCCGGAGACAGGTCGTAGCACCTGTGTAGATGAGAGAACTGAGTGCACAAAAAAAT	array('B', [7, 5, 10, 3, 7, 6, 4, 1, 5, 3, 3, 6, 4, 19, 12, 23, 18, 32, 27, 33, 9, 19, 30, 31, 26, 24, 13, 9, 14, 19, 15, 22, 15, 21, 34, 25, 30, 33, 24, 26, 29, 11, 22, 21, 29, 19, 16, 25, 12, 4, 4, 4, 5, 18, 16, 19, 22, 21, 14, 23, 20, 27, 23, 24, 26, 6, 23, 21, 28, 21, 22, 22, 22, 35, 18, 34, 21, 29, 14, 15, 12, 15, 16, 17, 19, 20, 19, 20, 17])	[('NM', 9), ('ms', 102), ('AS', 98), ('nn', 0), ('tp', 'P'), ('cm', 2), ('s1', 28), ('s2', 0), ('de', 0.07779999822378159), ('rl', 0)]
da8d2631-2eab-4c2a-8664-a792b09524e8	0	#0	10	3	89M	*	0	0	GTAGATAGGACTCTTTAGCTAGTGAACCACAGCCTCCGGAGACAGGTCGCGGCCTGTGTAGATGAGAGAACTGAGTGCACAAAAAAAAT	array('B', [3, 1, 15, 25, 28, 30, 17, 26, 20, 17, 8, 7, 13, 14, 4, 23, 24, 26, 27, 22, 8, 18, 21, 20, 16, 14, 14, 21, 4, 16, 17, 24, 21, 16, 17, 25, 22, 24, 31, 25, 16, 27, 23, 17, 8, 13, 13, 11, 14, 2, 

One line of a bam file corresponds to one read. It contains the following information:


1. **Query template NAME**: 88db3006-33a2-4019-b61b-000d67075abc
2. **bitwise FLAG**: 0 
3. **References sequence NAME**: oligo
4. **1- based leftmost mapping POSition**: 20
5. **MAPping Quality**: 60
6. **CIGAR string**: 6S22M1D56M     (https://genome.sph.umich.edu/wiki/SAM#What_is_a_CIGAR?)
7. **Ref. name of the mate/next read**: *       
8. **Position of the mate/next read**: 0       
9. **observed Template LENgth**: 0       
10. **segment SEQuence**: GATGGGCTCTTTAGCTAGTGAACCCTAGCACAGGAGACAGGTCGCGACCTGTGTAGATGAGAGAACTGAGTGCACAAAAAAAAT        
11. **ASCII of Phred-scaled base QUALity+33**: $$&*85+%%73;??82+16<2./7.,#(''*'428'%4=892/&#8682292@20?::63@6799995-545102011111-,,
12. **optional fields**: NM:i:4  ms:i:120   AS:i:120 nn:i:0  tp:A:P  cm:i:7  s1:i:68 s2:i:0  de:f:0.0506     rl:i:0

taken from https://samtools.github.io/hts-specs/SAMv1.pdf



## Pileup format

Pileup file was created using Samtools mpileup command:

```samtools mpileup -f Oligo_ref.fa -A -d 0 -Q 0 Oligo_1.bam > Oligo_1_a.pileup``` 

In [4]:
pileup = pd.read_csv("./Oligo_1.pileup", sep = "\t", header = None)
pileup.columns = ["chr name", "position", "ref base", "num reads", "read bases", "base qualities"]
pileup.head(10)

,chr name,position,ref base,num reads,read bases,base qualities
0,oligo,9,C,0,*,*
1,oligo,10,A,1,^+.,.
2,oligo,11,T,1,G,2
3,oligo,12,A,249,"......^+.^#.^$.^$.^#.^+.^#.^+.^$.^+.^#.^+.^,.^...",08002:././././.../.....//1././/.0././/.//.//.....
4,oligo,13,G,2713,.................................................,:9244352./4253244436/5<46103115/102:1:74167255...
5,oligo,14,A,3528,............................................+1...,=902.941219813902<159573925:..?5<2/6:62.54>172...
6,oligo,15,T,2711,........................-1A.............-1A......,?654<18417902.029:26372@6:6.:307:2621202241;86...
7,oligo,16,A,2017,.......................*.*...............*.......,27435374=1:?1;;;35;<=105;1.50768016/1/18.;<010...
8,oligo,17,G,5683,.................................................,;641;;6702.;9@94D/=45?>1?A<>470A.5.;03;.>82509...
9,oligo,18,G,4752,......................................-1A........,55513:>98@58?038/?2<??09A<907893.764660554.676...


**read bases** column:
- `.`: read matches reference (, on the reverse strand - in this case not so important because I work with RNA data)
- `[ACGT]`: read has mismatch a position ([acgt] on the reverse strand)
- `^`: first position covered by the read, this is followed by the alignment quality score:
    - quality score is given as an ASCII character to transform the character into a Phred quality score, subtract 33 from the the ASCII value.
    - For example: ^H corresponds to a reads that start at a given position with a mapping quality of `72(=H) - 33 = 39`
- `$`: last position covered by a given read
- `\+[0-9]+[ACGTNacgtn]+` (plus followed by a number and a series of ACGT): denotes insertion
    - \+ corresponds to insertion
    - [0-9] corresponds to the lenght of the insertion
    - [ACGTNacgtn] corresponds to the inserted sequence (upper case -> forward strand, lower case -> reverse strand) 
- `\-[0-9]+[ACGTNacgtn]+`: denotes deletion - same principle as insertion encoding
- `>`: denotes reference skip on forward strand (occurs when N is present in a CIGAR string; `<` on reverse strand)
- `*`: denotes a deletion where the deleted base(s) are not shown individually (is used if the specific bases of the deletion are not available)


**base qualities** column:
- each character corresponds to the base quality of a given read at the position
- ASCII value corresponds to the quality score (to get Phred score: subtract 33 from ASCII value)

In [12]:
# example read bases string
print(pileup.iloc[5, 4])
# example base quality string
print(pileup.iloc[5,5])

............................................+1T.............................................................................+1T..........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................-1T...............................................................................................................................+1T..........................................................................................................................

## Create functions 

- create function that extracts count (absolute, relative) of each base (A,C,G,T), number of insertions and number of deletions from a given pileup string (->`parse_pileup_string`)
- create function that calculates the `mismatch quality sum` (to compare to the coverage allele-fraction threshold) from a given line (-> `calc_mismatch_quality_sum`) 

Plan: remove one element after another from the string to extract all needed information.

In [8]:
pileup_string = ".C.+3AGTT.G...-2TTC..+3ATT..+1ATT.."

pattern = "(\+|\-)[0-9]+[ACGTNacgtn]+"
coords = []
for m in re.finditer(pattern, pileup_string):
    str_len = int(pileup_string[m.start(0)+1]) + 1
    coords.append((m.start(0), m.start(0)+1+str_len))

    print(pileup_string[m.start(0):m.start(0)+1+str_len])

print(list(reversed(coords))) # reverse list as to not shift the index downstream

for start, end in reversed(coords):
    pileup_string = pileup_string[:start] + pileup_string[end:]
print(pileup_string)

+3AGT
-2TT
+3ATT
+1A
[(28, 31), (21, 26), (14, 18), (3, 8)]
.C.T.G...C....TT..


In [4]:
def remove_indels(pileup_string: str) -> str:
    """
    Takes a pileup string and removes all occurences of the following patterns:
    '\+[0-9]+[ACGTNacgtn]+' for insertions
    '\-[0-9]+[ACGTNacgtn]+' for deletions

    Parameters
    ----------
    pileup_string : str
        Pileup string extracted from the fifth column of a pileup file

    Returns
    -------
    str
        Pileup strings with all occurences of the patterns above removed
    """
    pattern = "(\+|\-)[0-9]+[ACGTNacgtn]+"
    
    # get the start and end indices of all found patterns 
    coords = []
    for m in re.finditer(pattern, pileup_string):
        str_len = int(pileup_string[m.start(0)+1]) + 1
        coords.append((m.start(0), m.start(0)+1+str_len))
        
    # remove the patterns by the indices
    for start, end in reversed(coords): # reverse list as to not shift the index downstream
        pileup_string = pileup_string[:start] + pileup_string[end:]

    return pileup_string


def parse_pileup_string(pileup_string: str, ref_base: str) -> dict[int, int, int, int, int, int]:
    """
    Extracts the number of each base called at a given position, as well as the number
    of insertions and deletions. Information is extracted from a pileup string (fifth
    column in a pileup file).

    Parameters
    ----------
    pileup_string : str
        Pileup string extracted from the fifth column of a pileup file
    ref_base : str
        reference base at the position corresponding to the pileup string

    Returns
    -------
    dict
        Dictionary containing the number of A, T, C, G, 
        insertions and deletions.
    """

    pileup_string = pileup_string.lower()
    # remove all occurences of a caret and the following letter (could contain a,c,g,t)
    pileup_string = re.sub(r'\^.', '', pileup_string)

    ref_base = ref_base.lower()
    count_dict = {"a": 0, "t": 0, "c": 0, "g": 0, "ins": 0, "del": 0}

    # get number of insertions
    count_dict["ins"] = len(re.findall(r'\+[0-9]+[ACGTNacgtn]+', pileup_string))

    # get number of deletions
    count_dict["del"] = len(re.findall(r'\-[0-9]+[ACGTNacgtn]*|\*', pileup_string))

    # remove indel patterns to count the number of mismatches correctly
    pileup_string = remove_indels(pileup_string)

    # get number of mismatches (i.e. [ACGT])
    count_dict["a"] = pileup_string.count("a")
    count_dict["t"] = pileup_string.count("t")
    count_dict["c"] = pileup_string.count("c")
    count_dict["g"] = pileup_string.count("g")

    # get number of matches (determine where to count matches bases on ref_base)
    n_matches = pileup_string.count('.') + pileup_string.count(',')
    count_dict[ref_base] = n_matches

    return count_dict

def get_relative_count(count_dict: dict[int], n_reads: int) -> dict[float]:
    """
    Gets a dictionary containing the absolute counts for A, C, G and T 
    and calculates the relative proportions

    Parameters
    ----------
    count_dict : dict[int]
        Dictionary containing the absolute counts for A, C, G and T
    n_reads : int
        Number of reads at the given position

    Returns
    -------
    dict[float]
        Dictionary containing the relative counts for A, C, G and T
    """
    try:
        count_dict["a_rel"] = count_dict["a"] / n_reads
        count_dict["c_rel"] = count_dict["c"] / n_reads
        count_dict["g_rel"] = count_dict["g"] / n_reads
        count_dict["t_rel"] = count_dict["t"] / n_reads
    except ZeroDivisionError:
        count_dict["a_rel"] = 0
        count_dict["c_rel"] = 0
        count_dict["g_rel"] = 0
        count_dict["t_rel"] = 0

    return count_dict

def get_majority_base(count_dict: dict) -> str:
    """
    Gets a dictionary containing the absolute counts for A, C, G and T and returns the
    key of the one with the highest count.

    Parameters
    ----------
    count_dict : dict
        dictionary containing the absolute counts for A, C, G and T

    Returns
    -------
    str
        Key from the dictionary corresponding to the largest value
    """
    dict_subset = dict((k, count_dict[k]) for k in ("a", "c", "g", "t"))
    return max(dict_subset, key = dict_subset.get).upper()

def get_motif(chr: str, site: int, ref: str, k: int) -> str:
    """
    Extracts the motif of k bases up- and downstream from a given chromosomal site.
    Around the start and end of a refernce sequence the missing bases are filled with
    Ns.

    Parameters
    ----------
    chr : str
        name of the chromosome
    site : int
        position on the chromosome (1-indexed)
    ref : str
        reference sequence for the given chromosome 
    k : int
        number of bases to be regarded in both up- and downstream direction 
        
    Returns
    -------
    str
        sequence of k bases around the center site
    """ 
    idx = site-1
    n_ref = len(ref)

    if idx >= 0 and idx < n_ref:
        idx_l = idx-k
        idx_r = idx+k+1
        # left overhang
        if idx_l < 0:
            len_overhang = abs(idx_l)
            overhang = "N" * len_overhang
            motif = overhang + ref[:idx_r]
        # right overhang
        elif idx_r > n_ref:
            len_overhang = idx_r - n_ref
            overhang = "N" * len_overhang
            motif = ref[idx_l:] + overhang
        # no overhang
        else:
            motif = ref[idx_l:idx_r]

        return motif
    

def get_allele_fraction(counts: dict, ref_base: str) -> int:
    """
    Calculates the number of mismatched reads

    Parameters
    ----------
    counts : dict
        Dictionary containing the number of occurences of A,C,G,T for a given position
    ref_base : str
        reference base at the given position

    Returns
    -------
    int
        Number of mismatched reads a the given position
    """

    mismatch_count_sum = 0
    for b in ["a", "c", "g", "t"]:
        if b != ref_base.lower():
            mismatch_count_sum += counts[b+"_rel"]

    return mismatch_count_sum


def is_consensus_mismatch(threshold: float, counts: dict, n_reads: int, ref_base: str) -> bool:
    """
    At a given position, checks if the number of mismatched reads exceeds the given threshold.

    Parameters
    ----------
    threshold : float
        Corresponds to the percentage of mismatched reads that are 
        allowed before returning True
    counts : dict
        Dictionary containing the number of occurences of A,C,G,T for a given position
    n_reads : int
        Number of reads at the given position
    ref_base : str
        reference base at the given position

    Returns
    -------
    bool
        Does the number of mismatches at the position exceed the threshold? 

    """
    threshold = threshold * n_reads
    mismatch_count_sum = get_allele_fraction(counts, ref_base)
    return mismatch_count_sum >= threshold


def get_read_quality(read_qualities: str) -> tuple[float, float]:
    """
    Calculates the mean and std from the read qualities given in the sixth row
    of a pileup file.

    Parameters
    ----------
    read_qualities : str
        Read quality string from pileup file

    Returns
    -------
    tuple[float, float]
        Mean and standard deviation of read qualities
    """
    # transform string to list of corresponding phred numeric values
    vals = [code - 33 for code in read_qualities.encode("ascii")]

    mean = sum(vals)/len(vals)
    std = np.std(vals)

    return mean, std 


def get_references(path: str) -> dict[str]:
    """
    Reads a fasta file and stores it in a dictionary.

    Parameters
    ----------
    path : str
        filepath to a fasta file

    Returns
    -------
    dict[str]
        Dictionary where the key is the chromosome name and the value is the sequence
    """
    with open(path, "r") as ref:
        lines = ref.readlines()
        i = 0
        refs = {}
        for i in range(len(lines)):
            line = lines[i]
            if line.startswith(">"):
                refs[line[1:].strip()] = lines[i+1].strip()
        
    return refs

In [5]:
def process_position(line: list[str]) -> str:
    """
    Takes a line from a pileup file and processes it.

    Parameters
    ----------
    line : list[str]
        list containing each element from the pileup line.

    Returns
    -------
    str
        New line derived from the initial one. Can be written to a new file in consequent
        steps.
    """
    # extract elements from list
    chr, site, ref_base, n_reads, read_bases, read_qualities = line[0], int(line[1]), line[2], int(line[3]), line[4], line[5]
    
    # get reference sequence 
    ref = refs[chr]
    # get absolute number of A, C, G, T, ins, del
    count = parse_pileup_string(read_bases, ref_base)

    # get relative number of A, C, G and T counts
    count = get_relative_count(count, n_reads)

    # get majority base
    majority_base = get_majority_base(count)

    # get 11b motif
    motif = get_motif(chr, site, ref, k=5)

    # get allele fraction
    allele_fraction = get_allele_fraction(count, ref_base)

    # get qualitiy measures
    quality_mean, quality_std = get_read_quality(read_qualities)

    out = f'{chr}\t{site}\t{n_reads}\t{ref_base}\t{majority_base}\t{count["a"]}\t{count["c"]}\t{count["g"]}\t{count["t"]}\t{count["a_rel"]}\t{count["c_rel"]}\t{count["g_rel"]}\t{count["t_rel"]}\t{motif}\t{allele_fraction}\t{quality_mean}\t{quality_std}\n'
    return out

def process_file(infile: str, outfile: str, ref: str):
    """
    Reads .pileup file and processes it, writing the results to a new file.

    Parameters
    ----------
    infile : str
        path to the input .pileup file
    outfile : str
        path to the output tsv file
    ref : str
        path to the reference fasta
        
    Returns
    -------
    None
    """
    refs = get_references(ref)

    with open(infile, "r") as i:
        lines = i.readlines()

        with open(outfile, "w") as o:
            header = f"chr\tsite\tn_reads\tref_base\tmajority_base\tn_a\tn_c\tn_g\tn_t\tn_a_rel\tn_c_rel\tn_g_rel\tn_t_rel\tmotif\tperc_mismatched\tq_mean\tq_std\n"
            o.write(header)
            for line in lines:
                outline = process_position(line.split("\t"))
                o.write(outline)

In [7]:
refs = get_references("./Oligo_ref.fa")
process_file("./Oligo_1.pileup", "./Oligo_1_out.tsv", "./Oligo_ref.fa")

In [77]:
refs = get_references("./curlcake_ref.fa")
process_file("./curlcake_m6a.pileup", "./curlcake_m6a_out.tsv", "./curlcake_ref.fa")